In [98]:
import re

import pandas as pd
from rapidfuzz import fuzz

from copy import copy
from scipy import stats
import warnings
warnings.simplefilter('ignore')

import numpy as np
from statsmodels.api import add_constant, OLS

In [104]:
df = pd.read_excel('./eu_sunctions.xlsx')
df = df.drop(columns='Unnamed: 0')
df = df.dropna()
df = df[df['Name'] != 'Name']
df = df[df['Name'] != 'Names']
df = df[df['Identifying information'].str.contains('DOB:') == False]
df = df[df['Identifying information'].str.contains('DOB') == False]
df = df[df['Identifying information'].str.contains('Associated individuals:') == False]
df = df[df['Identifying information'].str.contains('Gender:') == False]
df = df[df['Identifying information'].str.contains('Gender') == False]
df = df[df['Identifying information'].str.contains('Nationality:') == False]
df = df[df['Identifying information'].str.contains('Nationality') == False]

df.to_excel('./eu_sanctions_entities.xlsx')
df

,Name,Identifying information,Reasons,Date of listing
501,Gas Industry Insurance Company SOGAZ\n(Акционе...,"Address: Sakharov boulevard 10, Moscow, 107078...","Address: Sakharov boulevard 10, Moscow, 107078...",Telephone:
723,State Unitary Enterprise of the ‘Republic of C...,"Prospekt Kirov 52, Simferopol, Crimea, Ukraine...","Prospekt Kirov 52, Simferopol, Crimea, Ukraine...","пр. Кирова 52, г. Симферополь, Крым, Украина 2..."
726,Federal state budget institution for science a...,"298600, Kirov Street 31, Yalta, Crimea, Ukrain...","298600, Kirov Street 31, Yalta, Crimea, Ukraine","298600, ул. Кирова, 31, г. Ялта, Крым, Украина"
730,Joint-stock company ‘Sparkling wine plant “Nov...,"298032, Crimea, Sudak, Novy Svet, str. Shalapi...","298032, Crimea, Sudak, Novy Svet, str. Shalapi...","298032, Крым, г. Судак, пгт. Новый Свет, ул. Ш..."
734,Public movement ‘Peace to Luhansk Region’ (Mir...,https://mir-lug.info/\nAddress: Karl Marx Stre...,Public ‘organisation’ that presented candidate...,29.11.2014
...,...,...,...,...
3171,Joint-stock company “PMC Wagner Centre”\n(ЧВК ...,"Address: 15 Zolnaya Street, Building 1, Room 1...",PMC Wagner Centre is a Russian commercial vent...,23.6.2023
3173,Akuta JSC\na.k.a. Acuta\na.k.a. Zakrytoe Aktsi...,Place of registration: Russian Federation (197...,Akuta JSC holds a license administered by the ...,23.6.2023
3174,20th Guards Combined Arms Army\n(20-я гвардейс...,Place of registration: Russian Federation (606...,The 20th Guards Combined Arms Army is a part o...,23.6.2023
3175,53rd Anti-Aircraft Missile Brigade\na.k.a. 53r...,Place of registration: Russian Federation (305...,The 53rd Anti-Aircraft Missile Brigade is a su...,23.6.2023


In [111]:
df = pd.read_excel('./eu_sanctions_entities.xlsx')
# Преобразование столбца с датами в формат datetime
df['Date of listing'] = pd.to_datetime(df['Date of listing'], format='%d.%m.%Y')

# Удаление строк, где значение даты меньше 01.01.2022
df = df[df['Date of listing'] >= '2022-01-01']
df

,Unnamed: 0,Name,Identifying information,Reasons,Date of listing
0,501,Gas Industry Insurance Company SOGAZ\n(Акционе...,"Address: Sakharov boulevard 10, Moscow, 107078...","Address: Sakharov boulevard 10, Moscow, 107078...",2022-04-03
3,761,ROSNEFT AERO (RN AERO) (Russian: Роснефть-Аэро...,"Address: 15, Malaya Kaluzhskaya Str., Moscow, ...",Rosneft Aero delivers jet fuel to the Simferop...,2022-03-15
4,762,JSC ROSOBORONEXPORT (Russian: AO Рособоронэксп...,"Address: 27 Stromynka Str., Moscow, 107076, Ru...",Rosoboronexport is Russia’s only official stat...,2022-03-15
5,763,JSC NPO High Precision Systems (Russian: АО НП...,"Address: 7 Kievskaya Str., Moscow, 121059, Rus...",High Precision Systems is a Russian developer ...,2022-03-15
6,764,JSC Kurganmashzavod (Russian: ПАО Курганмашзавод),"Address: 17 1J Mashinostroitely Ave., 640021, ...",Kurganmashzavod is a major Russian military-in...,2022-03-15
...,...,...,...,...,...
420,3171,Joint-stock company “PMC Wagner Centre”\n(ЧВК ...,"Address: 15 Zolnaya Street, Building 1, Room 1...",PMC Wagner Centre is a Russian commercial vent...,2023-06-23
421,3173,Akuta JSC\na.k.a. Acuta\na.k.a. Zakrytoe Aktsi...,Place of registration: Russian Federation (197...,Akuta JSC holds a license administered by the ...,2023-06-23
422,3174,20th Guards Combined Arms Army\n(20-я гвардейс...,Place of registration: Russian Federation (606...,The 20th Guards Combined Arms Army is a part o...,2023-06-23
423,3175,53rd Anti-Aircraft Missile Brigade\na.k.a. 53r...,Place of registration: Russian Federation (305...,The 53rd Anti-Aircraft Missile Brigade is a su...,2023-06-23


In [112]:
def remove_russian(text):
    return re.sub(r'[а-яА-Я]', '', text)

def remove_russian_text(text):
    pattern = r'(.*?)\s\(Russian:\s*\)'
    match = re.search(pattern, text)
    if match:
        return match.group(1)
    else:
        return text


df['Name'] = df['Name'].apply(remove_russian)
df['Name'] = df['Name'].apply(remove_russian_text)
df['Name'] = df['Name'].str.replace(r'\(.*\)', '', regex=True)
df['Name'] = df['Name'].str.replace(r'\s*\(.*\)', '', regex=True)
df['Name'] = df['Name'].str.replace(r'\(Russian:.*\)', '', regex=True)

df

,Unnamed: 0,Name,Identifying information,Reasons,Date of listing
0,501,Gas Industry Insurance Company SOGAZ\n,"Address: Sakharov boulevard 10, Moscow, 107078...","Address: Sakharov boulevard 10, Moscow, 107078...",2022-04-03
3,761,ROSNEFT AERO,"Address: 15, Malaya Kaluzhskaya Str., Moscow, ...",Rosneft Aero delivers jet fuel to the Simferop...,2022-03-15
4,762,JSC ROSOBORONEXPORT,"Address: 27 Stromynka Str., Moscow, 107076, Ru...",Rosoboronexport is Russia’s only official stat...,2022-03-15
5,763,JSC NPO High Precision Systems,"Address: 7 Kievskaya Str., Moscow, 121059, Rus...",High Precision Systems is a Russian developer ...,2022-03-15
6,764,JSC Kurganmashzavod,"Address: 17 1J Mashinostroitely Ave., 640021, ...",Kurganmashzavod is a major Russian military-in...,2022-03-15
...,...,...,...,...,...
420,3171,Joint-stock company “PMC Wagner Centre”\n,"Address: 15 Zolnaya Street, Building 1, Room 1...",PMC Wagner Centre is a Russian commercial vent...,2023-06-23
421,3173,Akuta JSC\na.k.a. Acuta\na.k.a. Zakrytoe Aktsi...,Place of registration: Russian Federation (197...,Akuta JSC holds a license administered by the ...,2023-06-23
422,3174,20th Guards Combined Arms Army\n(20- \na.k....,Place of registration: Russian Federation (606...,The 20th Guards Combined Arms Army is a part o...,2023-06-23
423,3175,53rd Anti-Aircraft Missile Brigade\na.k.a. 53r...,Place of registration: Russian Federation (305...,The 53rd Anti-Aircraft Missile Brigade is a su...,2023-06-23


In [113]:
df.to_excel('./eu_sanctions_entities_very_clear.xlsx')

In [4]:
df = pd.read_excel('./eu_sanctions_entities_very_clear.xlsx')
columns = [
    'Unnamed: 0.1',
    'Unnamed: 0'
]
df = df.drop(columns= columns)
df

,Name,Identifying information,Reasons,Date of listing
0,Gas Industry Insurance Company SOGAZ\n,"Address: Sakharov boulevard 10, Moscow, 107078...","Address: Sakharov boulevard 10, Moscow, 107078...",2022-04-03
1,ROSNEFT AERO,"Address: 15, Malaya Kaluzhskaya Str., Moscow, ...",Rosneft Aero delivers jet fuel to the Simferop...,2022-03-15
2,JSC ROSOBORONEXPORT,"Address: 27 Stromynka Str., Moscow, 107076, Ru...",Rosoboronexport is Russia’s only official stat...,2022-03-15
3,JSC NPO High Precision Systems,"Address: 7 Kievskaya Str., Moscow, 121059, Rus...",High Precision Systems is a Russian developer ...,2022-03-15
4,JSC Kurganmashzavod,"Address: 17 1J Mashinostroitely Ave., 640021, ...",Kurganmashzavod is a major Russian military-in...,2022-03-15
...,...,...,...,...
397,Joint-stock company “PMC Wagner Centre”\n,"Address: 15 Zolnaya Street, Building 1, Room 1...",PMC Wagner Centre is a Russian commercial vent...,2023-06-23
398,Akuta JSC\na.k.a. Acuta\na.k.a. Zakrytoe Aktsi...,Place of registration: Russian Federation (197...,Akuta JSC holds a license administered by the ...,2023-06-23
399,20th Guards Combined Arms Army\n(20- \na.k....,Place of registration: Russian Federation (606...,The 20th Guards Combined Arms Army is a part o...,2023-06-23
400,53rd Anti-Aircraft Missile Brigade\na.k.a. 53r...,Place of registration: Russian Federation (305...,The 53rd Anti-Aircraft Missile Brigade is a su...,2023-06-23


In [125]:
df.to_csv('./sanction_entities_EU.csv')

In [114]:
df_moex = pd.read_excel('./members_moex.xlsx')
df_moex = df_moex['Short name of Trading Member']
df_moex = pd.DataFrame(df_moex)
df_moex = df_moex.dropna()
df_moex.to_excel('./members_moex_names.xlsx')


In [115]:
df_moex = df_moex.rename(columns={'Short name of Trading Member': 'Name'})
df_moex

,Name
4,Waybank JSC
5,"""ZIRAAT BANK (MOSCOW)"" (JSC)"
6,"""Kaspi Bank"" JSC"
7,"JSC ""Tansar Capital"""
8,Absolut Bank (PAO)
...,...
615,"""GPB-factoring"" (LLC)"
616,"OJSC ""BAKAI BANK"""
617,"JSC ""Technobank"""
618,Credit Union ASIAPAY LLC


In [36]:
print(fuzz.ratio('Sberbank(Сбербанк)', "Sber bank"))

59.25925925925925


In [116]:

# Функция для сравнения строк
def compare_strings(string1, string2):
    return fuzz.ratio(string1, string2)  # Возвращает коэффициент сходства строк


# Проход по каждой строке в первом датасете
for index1, row1 in df.iterrows():
    match_found = False
    for index2, row2 in df_moex.iterrows():
        similarity = compare_strings(row1['Name'], row2['Name'])
        if similarity >= 50:  # Порог сходства в 50%
            match_found = True
            break

    if not match_found:
        # Удалить данные о компании из первого датасета
        df.drop(index1, inplace=True)

# Сохранение обновленного первого датасета
df

,Unnamed: 0,Name,Identifying information,Reasons,Date of listing
0,501,Gas Industry Insurance Company SOGAZ\n,"Address: Sakharov boulevard 10, Moscow, 107078...","Address: Sakharov boulevard 10, Moscow, 107078...",2022-04-03
3,761,ROSNEFT AERO,"Address: 15, Malaya Kaluzhskaya Str., Moscow, ...",Rosneft Aero delivers jet fuel to the Simferop...,2022-03-15
4,762,JSC ROSOBORONEXPORT,"Address: 27 Stromynka Str., Moscow, 107076, Ru...",Rosoboronexport is Russia’s only official stat...,2022-03-15
5,763,JSC NPO High Precision Systems,"Address: 7 Kievskaya Str., Moscow, 121059, Rus...",High Precision Systems is a Russian developer ...,2022-03-15
6,764,JSC Kurganmashzavod,"Address: 17 1J Mashinostroitely Ave., 640021, ...",Kurganmashzavod is a major Russian military-in...,2022-03-15
...,...,...,...,...,...
398,3049,JSC MZ RIP,Type of entity: Joint Stock Company\nRegistrat...,By Presidential Decree No. 412 of 23 April 202...,2024-02-23
399,3050,JSC DNPP,Type of entity: Joint Stock Company\nRegistrat...,By Presidential Decree No. 412 of 23 April 202...,2024-02-23
402,3053,LLC NOVELCO\na.k.a.\nLimited Liability company...,"Address: 125493 Russian Federation, Moscow, Sm...",LLC Novelco is a logistics company established...,2024-02-23
419,3169,JSC GTLK\nState Transport Leasing Company,"Address: 31a Leningradsky Avenue, bldg. #1, Mo...",JSC GTLK is a Russian state-owned enterprise o...,2022-04-08


In [ ]:
df = df[['Name', '']]

In [117]:
df.to_csv('./List_of_sunction_companies_from_moex_EU.csv')

In [109]:
df = pd.read_excel('./List_of_sunction_companies_from_moex_EU.xlsx')
columns = [
    'Name',
    'Date of listing'
]
df = df[columns]
df.to_excel('./List_of_sunction_companies_from_moex_EU_2clmn.xlsx')



In [136]:
df_us = pd.read_csv('./Data for diploma - US sanctioned companies.csv', header= None)
df_us = df_us.drop([0,0])
df_us.columns = df_us.iloc[0]

df_us

1,24.02.2022,28.02.2022,11.03.2022,24.03.2022,31.03.2022,06.04.2022,07.04.2022,20.04.2022,28.06.2022,02.08.2022,...,12.04.2023,19.05.2023,19.05.2023,27.06.2023,20.07.2023,14.09.2023,02.11.2023,12.12.2023,20.12.2023,23.02.2024
1,24.02.2022,28.02.2022,11.03.2022,24.03.2022,31.03.2022,06.04.2022,07.04.2022,20.04.2022,28.06.2022,02.08.2022,...,12.04.2023,19.05.2023,19.05.2023,27.06.2023,20.07.2023,14.09.2023,02.11.2023,12.12.2023,20.12.2023,23.02.2024
2,Sberbank,JSC RDIF,VTB Bank,Tactical Missiles Corporation JSC (KTRV),OOO Serniya Engineering,Subsidiary Bank Sberbank of Russia JSC,Alrosa,Transkapitalbank (TKB),Rostec,MMK-FINANS,...,Windfel,Rosatom,Trust Logistic,Diamville,Radiotekhsnab (RTS),Mayak,Absolyut Shar,Izhevsk Electromechanical Plant Kupol (AO IEMZ...,SUN Ship,Albatros
3,VTB Bank,LLC RVC,NaN,Globus Ryazan Design Bureau JSC,OOO Sertal,JSC Sberbank,USC,Investtradebank,United Aircraft Corporation,MMK Metalurji,...,Savoler,Hydrographic Company,Versvet,Industrial Resources,Progress Lider,Transmashholding JSC (Transmash),Alfa,Aerokon,Sovcomflot,Drake
4,Otkritie,NaN,NaN,JSC Ural Design Bureau Detal,OOO Robin Treid,Open JSC BPS-Sberbank,NaN,MarGlo,Tupolev,JSC PPIT,...,Miramonte,Tagras Holding,Ostec,NaN,GTME Tekhnologii,Agat,FAM Grupp,TSK Vektor,Bellatrix,Alabuga
5,Sovcombank,NaN,NaN,Temp-Avia Arzamas Research and Production Asso...,OOO Nauchno-Tekhnicheskii Tsentr Metrotek,Auction LLC,NaN,Tsargrad,KAPO-Avtotrans,NaN,...,KTH Group,Bashnipineft,Avesto,NaN,Region-Prof,Siberica Oy (Siberica),Gazpromneft Catalytic Systems,JSC 75 Arsenal,Covart,National Payment Card System JSC (NSPK)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,LLC Nissa Digispace,NaN,NaN
74,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Picaso 3D LLC,NaN,NaN
75,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,LLC Tsvetnoy Mir,NaN,NaN
76,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Expobank,NaN,NaN


OA, AO, OOO, Joint Stock commercial bank, Joint-stock company, Joint Stock Company, Limited Liability Company, Obshestvo S Ogranichennoj Otvetstvennostyu, Otkrytoe Aktsionernoe Obschestvo, OAO, LLC, Limited, Ltd. , Open, Commercial, Public, JSC, PJSC, NPP, OJSC, OPEN JOINT STOCK COMPANY, PUBLIC JOINT STOCK COMPANY, Open Joint Stock Company, Public Joint Stock Company, Aktsionernoe Obschestvo, LIMITED LIABILITY COMPANY, Joint-stock commercial bank, Joint Stock Commercial Bank, Open JSC, Public JSC

In [2]:
df = pd.read_csv('./sanction_entities_EU.csv')

df

NameError: name 'pd' is not defined

, , LIMITED LIABILITY COMPANY, , , ,

In [157]:

df['Name'] = df['Name'].str.replace('NPO', '')
df['Name'] = df['Name'].str.replace('LLC', '')
df['Name'] = df['Name'].str.replace('OA', '')
df['Name'] = df['Name'].str.replace('AO', '')
df['Name'] = df['Name'].str.replace('OOO', '')
df['Name'] = df['Name'].str.replace('Joint Stock commercial bank', '')
df['Name'] = df['Name'].str.replace('Joint-stock company', '')
df['Name'] = df['Name'].str.replace('Joint Stock Company', '')
df['Name'] = df['Name'].str.replace('Limited Liability Company', '')
df['Name'] = df['Name'].str.replace('Obshestvo S Ogranichennoj Otvetstvennostyu', '')
df['Name'] = df['Name'].str.replace('Otkrytoe Aktsionernoe Obschestvo', '')
df['Name'] = df['Name'].str.replace('OAO', '')
df['Name'] = df['Name'].str.replace('LLC', '')
df['Name'] = df['Name'].str.replace('Limited', '')
df['Name'] = df['Name'].str.replace('Ltd.', '')
df['Name'] = df['Name'].str.replace('Commercial', '')
df['Name'] = df['Name'].str.replace('Ltd', '')
df['Name'] = df['Name'].str.replace('Public', '')
df['Name'] = df['Name'].str.replace('PJSC', '')
df['Name'] = df['Name'].str.replace('OJSC', '')
df['Name'] = df['Name'].str.replace('OPEN JOINT STOCK COMPANY', '')
df['Name'] = df['Name'].str.replace('PUBLIC JOINT STOCK COMPANY', '')
df['Name'] = df['Name'].str.replace('Open Joint Stock Company', '')
df['Name'] = df['Name'].str.replace('Public Joint Stock Company', '')
df['Name'] = df['Name'].str.replace('Aktsionernoe Obschestvo', '')
df['Name'] = df['Name'].str.replace('LIMITED LIABILITY COMPANY', '')
df['Name'] = df['Name'].str.replace('Joint-stock commercial bank', '')
df['Name'] = df['Name'].str.replace('Joint Stock Commercial Bank', '')
df['Name'] = df['Name'].str.replace('Open JSC', '')
df['Name'] = df['Name'].str.replace('Public JSC', '')
df['Name'] = df['Name'].str.replace('JSC', '')

df_test = df.copy()
df

,Name,Identifying information,Reasons,Date of listing
0,Gas Industry Insurance Company SOGAZ\n,"Address: Sakharov boulevard 10, Moscow, 107078...","Address: Sakharov boulevard 10, Moscow, 107078...",2022-04-03
1,ROSNEFT AERO,"Address: 15, Malaya Kaluzhskaya Str., Moscow, ...",Rosneft Aero delivers jet fuel to the Simferop...,2022-03-15
2,ROSOBORONEXPORT,"Address: 27 Stromynka Str., Moscow, 107076, Ru...",Rosoboronexport is Russia’s only official stat...,2022-03-15
3,High Precision Systems,"Address: 7 Kievskaya Str., Moscow, 121059, Rus...",High Precision Systems is a Russian developer ...,2022-03-15
4,Kurganmashzavod,"Address: 17 1J Mashinostroitely Ave., 640021, ...",Kurganmashzavod is a major Russian military-in...,2022-03-15
...,...,...,...,...
397,“PMC Wagner Centre”\n,"Address: 15 Zolnaya Street, Building 1, Room 1...",PMC Wagner Centre is a Russian commercial vent...,2023-06-23
398,Akuta \na.k.a. Acuta\na.k.a. Zakrytoe Aktsione...,Place of registration: Russian Federation (197...,Akuta JSC holds a license administered by the ...,2023-06-23
399,20th Guards Combined Arms Army\n(20- \na.k....,Place of registration: Russian Federation (606...,The 20th Guards Combined Arms Army is a part o...,2023-06-23
400,53rd Anti-Aircraft Missile Brigade\na.k.a. 53r...,Place of registration: Russian Federation (305...,The 53rd Anti-Aircraft Missile Brigade is a su...,2023-06-23


In [163]:
df.to_csv('./sanction_entities_EU.csv')

In [162]:
df_test = df.copy()

def clean_string(input_string):
    return input_string.split(" ")[0]
def remove_russian_text(text):
    pattern = r'(.*?)\s\()\s*\)'
    match = re.search(pattern, text)
    if match:
        return match.group(1)
    else:
        return text


df_test['Name'] = df_test['Name'].apply(remove_russian)

df_test

,Name,Identifying information,Reasons,Date of listing
0,Gas Industry Insurance Company SOGAZ\n,"Address: Sakharov boulevard 10, Moscow, 107078...","Address: Sakharov boulevard 10, Moscow, 107078...",2022-04-03
1,ROSNEFT AERO,"Address: 15, Malaya Kaluzhskaya Str., Moscow, ...",Rosneft Aero delivers jet fuel to the Simferop...,2022-03-15
2,ROSOBORONEXPORT,"Address: 27 Stromynka Str., Moscow, 107076, Ru...",Rosoboronexport is Russia’s only official stat...,2022-03-15
3,High Precision Systems,"Address: 7 Kievskaya Str., Moscow, 121059, Rus...",High Precision Systems is a Russian developer ...,2022-03-15
4,Kurganmashzavod,"Address: 17 1J Mashinostroitely Ave., 640021, ...",Kurganmashzavod is a major Russian military-in...,2022-03-15
...,...,...,...,...
397,“PMC Wagner Centre”\n,"Address: 15 Zolnaya Street, Building 1, Room 1...",PMC Wagner Centre is a Russian commercial vent...,2023-06-23
398,Akuta \na.k.a. Acuta\na.k.a. Zakrytoe Aktsione...,Place of registration: Russian Federation (197...,Akuta JSC holds a license administered by the ...,2023-06-23
399,20th Guards Combined Arms Army\n(20- \na.k....,Place of registration: Russian Federation (606...,The 20th Guards Combined Arms Army is a part o...,2023-06-23
400,53rd Anti-Aircraft Missile Brigade\na.k.a. 53r...,Place of registration: Russian Federation (305...,The 53rd Anti-Aircraft Missile Brigade is a su...,2023-06-23


In [146]:
df_us = pd.read_csv('./Data for diploma - US sanctioned companies.csv', header= None)
df_us = df_us.drop([0,0])
df_us.columns = df_us.iloc[0]

df_us

1,24.02.2022,28.02.2022,11.03.2022,24.03.2022,31.03.2022,06.04.2022,07.04.2022,20.04.2022,28.06.2022,02.08.2022,...,12.04.2023,19.05.2023,19.05.2023,27.06.2023,20.07.2023,14.09.2023,02.11.2023,12.12.2023,20.12.2023,23.02.2024
1,24.02.2022,28.02.2022,11.03.2022,24.03.2022,31.03.2022,06.04.2022,07.04.2022,20.04.2022,28.06.2022,02.08.2022,...,12.04.2023,19.05.2023,19.05.2023,27.06.2023,20.07.2023,14.09.2023,02.11.2023,12.12.2023,20.12.2023,23.02.2024
2,Sberbank,JSC RDIF,VTB Bank,Tactical Missiles Corporation JSC (KTRV),OOO Serniya Engineering,Subsidiary Bank Sberbank of Russia JSC,Alrosa,Transkapitalbank (TKB),Rostec,MMK-FINANS,...,Windfel,Rosatom,Trust Logistic,Diamville,Radiotekhsnab (RTS),Mayak,Absolyut Shar,Izhevsk Electromechanical Plant Kupol (AO IEMZ...,SUN Ship,Albatros
3,VTB Bank,LLC RVC,NaN,Globus Ryazan Design Bureau JSC,OOO Sertal,JSC Sberbank,USC,Investtradebank,United Aircraft Corporation,MMK Metalurji,...,Savoler,Hydrographic Company,Versvet,Industrial Resources,Progress Lider,Transmashholding JSC (Transmash),Alfa,Aerokon,Sovcomflot,Drake
4,Otkritie,NaN,NaN,JSC Ural Design Bureau Detal,OOO Robin Treid,Open JSC BPS-Sberbank,NaN,MarGlo,Tupolev,JSC PPIT,...,Miramonte,Tagras Holding,Ostec,NaN,GTME Tekhnologii,Agat,FAM Grupp,TSK Vektor,Bellatrix,Alabuga
5,Sovcombank,NaN,NaN,Temp-Avia Arzamas Research and Production Asso...,OOO Nauchno-Tekhnicheskii Tsentr Metrotek,Auction LLC,NaN,Tsargrad,KAPO-Avtotrans,NaN,...,KTH Group,Bashnipineft,Avesto,NaN,Region-Prof,Siberica Oy (Siberica),Gazpromneft Catalytic Systems,JSC 75 Arsenal,Covart,National Payment Card System JSC (NSPK)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,LLC Nissa Digispace,NaN,NaN
74,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Picaso 3D LLC,NaN,NaN
75,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,LLC Tsvetnoy Mir,NaN,NaN
76,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Expobank,NaN,NaN


In [165]:
df_us = pd.read_csv('./Data for diploma - US sanctioned companies_new.csv')


df_us

,Company,Date
0,Sberbank,24.02.2022
1,VTB Bank,24.02.2022
2,Otkritie,24.02.2022
3,Sovcombank,24.02.2022
4,Novikombank,24.02.2022
...,...,...
524,Modulbank,23.02.2024
525,Databank,23.02.2024
526,Maritime,23.02.2024
527,Bystrobank,23.02.2024


# testing weak-form efficiency

In [3]:
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf
import numpy as np


In [51]:
# Data processing
df = pd.read_csv('./stock_price.csv')
df = df [['TRADEDATE','CLOSE', 'TICKER']]
df2 = pd.read_excel('IMOEX.xlsx')

# df2['TRADEDATE'] = df2['TRADEDATE'].astype(str).str.replace('.', '-')
# # df['TRADEDATE'] = pd.to_datetime(df['TRADEDATE'])
df2['TRADEDATE'] = pd.to_datetime(df2['TRADEDATE'],format='%d.%m.%Y' )
df= pd.concat([df, df2])
df['CLOSE'] = pd.to_numeric(df['CLOSE'], errors='coerce')
# df['VOLUME'] = pd.to_numeric(df['VOLUME'], errors='coerce')
# df['VOLUME'] = df['VOLUME'].fillna(0)
# df['CLOSE'] = df['CLOSE'].fillna()



df = df.loc[df['TRADEDATE'] != 'TRADEDATE']
df = df.dropna()
df['TRADEDATE'] = pd.to_datetime(df['TRADEDATE'])


df = df.sort_values(by='TRADEDATE', ascending=True)

df

,TRADEDATE,CLOSE,TICKER
17317,2021-02-01,35.2350,RUAL
49124,2021-02-01,88.4000,NKNC
45772,2021-02-01,70.3400,AFLT
59471,2021-02-01,0.1544,MRKU
20793,2021-02-01,464.0000,MDMG
...,...,...,...
30671,2024-05-17,1679.0000,TRNFP
29833,2024-05-17,746.5000,TATN
58631,2024-05-17,22.8700,LSNG
49122,2024-05-17,16336.0000,AKRN


### Weak-form efficiency 

In [52]:
# Calculate returns with price on stock
data = df.pivot(index='TRADEDATE', columns='TICKER', values='CLOSE')
print(data.isnull().sum())
# data = data.dropna(axis=1, thresh=len(data) - 10)
data = data.fillna(method='ffill').fillna(method='bfill')
returns = data
returns

TICKER
AFKS      1
AFLT      1
AGRO      5
AKRN      3
ALRS      1
       ... 
VKCO    250
VSMO      3
VTBR      1
WUSH    458
YNDX      4
Length: 85, dtype: int64


/tmp/ipykernel_442323/2596952430.py:5: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data = data.fillna(method='ffill').fillna(method='bfill')


TICKER,AFKS,AFLT,AGRO,AKRN,ALRS,AQUA,ASTR,BANE,BSPB,CARM,...,TRMK,TRNFP,UGLD,UPRO,VEON-RX,VKCO,VSMO,VTBR,WUSH,YNDX
TRADEDATE,,,,,,,,,,,,,,,,,,,,,
2021-02-01,32.475,70.34,897.0,6062.0,100.61,249.0,466.20,1636.5,53.74,3.3180,...,59.30,143050.0,0.5710,2.844,139.00,897.4,21780.0,0.036945,185.40,4847.4
2021-02-02,33.316,70.14,902.8,6096.0,102.15,246.5,466.20,1660.0,54.37,3.3180,...,59.40,143350.0,0.5710,2.829,139.00,897.4,21820.0,0.036945,185.40,4997.0
2021-02-03,33.650,70.68,889.4,6102.0,103.66,249.0,466.20,1665.5,54.28,3.3180,...,58.24,144100.0,0.5710,2.848,139.00,897.4,21780.0,0.037000,185.40,4917.2
2021-02-04,33.963,70.56,894.0,6122.0,102.61,250.0,466.20,1655.0,55.08,3.3180,...,58.40,143900.0,0.5710,2.871,139.00,897.4,21980.0,0.037380,185.40,5016.6
2021-02-05,34.992,70.82,893.0,6090.0,102.32,251.0,466.20,1659.0,54.55,3.3180,...,58.74,144750.0,0.5710,2.861,139.00,897.4,22040.0,0.037885,185.40,4973.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-05-13,27.038,52.11,1545.4,16936.0,79.58,903.5,649.60,3401.0,332.10,2.3495,...,225.46,1643.0,0.9857,2.204,39.50,577.6,39520.0,0.023100,304.33,4407.0
2024-05-14,27.999,54.19,1499.0,16800.0,79.23,901.0,643.55,3395.0,332.80,2.3560,...,224.90,1671.5,0.9932,2.189,39.10,603.6,39460.0,0.023060,322.80,4489.8
2024-05-15,28.980,56.75,1517.8,16772.0,79.70,901.0,640.35,3378.0,339.73,2.4070,...,223.46,1667.5,1.0011,2.153,38.85,616.4,39440.0,0.023205,314.62,4355.2


In [53]:
import pandas as pd
from statsmodels.tsa.stattools import adfuller

# Функция для выполнения теста ADF и возвращения результатов в виде Series
def adf_test(series):
    result = adfuller(series.dropna(), autolag='AIC')
    labels = ['ADF Test Statistic', 'p-value', '#Lags Used', 'Number of Observations Used']
    out = pd.Series(result[0:4], index=labels)
    for key, value in result[4].items():
        out[f'Critical Value ({key})'] = value
    return out

# Сбор всех результатов в один DataFrame
results_dict = {}

for company in returns.columns:
    results_dict[company] = adf_test(returns[company])

# Преобразование словаря результатов в DataFrame
results_df = pd.DataFrame(results_dict)

# Транспонирование DataFrame для желаемого формата (компании в столбцах)
results_df = results_df.T

# Добавление столбца с результатом гипотезы
results_df['null hypothesis'] = results_df['p-value'].apply(lambda p: 'reject' if p <= 0.05 else 'not reject')

# Вывод таблицы
print(results_df)

# Вывод интерпретации результатов (опционально, если требуется дополнительное пояснение)
for company in returns.columns:
    p_value = results_df.loc[company, 'p-value']
    if p_value <= 0.05:
        print(f"{company}: Strong evidence against the null hypothesis, reject the null hypothesis. Data has no unit root and is stationary.")
    else:
        print(f"{company}: Weak evidence against the null hypothesis, time series has a unit root, indicating it is non-stationary.")



      ADF Test Statistic   p-value  #Lags Used  Number of Observations Used  \
AFKS           -1.608600  0.479272         4.0                        815.0   
AFLT           -1.381023  0.591340         9.0                        810.0   
AGRO           -1.078828  0.723441         0.0                        819.0   
AKRN           -1.763257  0.398850        20.0                        799.0   
ALRS           -0.938766  0.774938        11.0                        808.0   
...                  ...       ...         ...                          ...   
VKCO           -1.507248  0.529923         7.0                        812.0   
VSMO           -1.925412  0.320203        10.0                        809.0   
VTBR           -1.103302  0.713753        15.0                        804.0   
WUSH            0.047429  0.962283        20.0                        799.0   
YNDX           -1.368300  0.597376         7.0                        812.0   

      Critical Value (1%)  Critical Value (5%)  Cri

In [23]:
results_df

,ADF Test Statistic,p-value,#Lags Used,Number of Observations Used,Critical Value (1%),Critical Value (5%),Critical Value (10%),null hypothesis
AFKS,-1.383072,0.590364,2.0,816.0,-3.438389,-2.865088,-2.568660,not reject
AFLT,-1.380070,0.591793,9.0,809.0,-3.438459,-2.865119,-2.568676,not reject
AGRO,-1.078918,0.723405,0.0,814.0,-3.438409,-2.865097,-2.568664,not reject
AKRN,-1.767529,0.396698,15.0,801.0,-3.438540,-2.865155,-2.568695,not reject
ALRS,-0.939516,0.774681,11.0,807.0,-3.438479,-2.865128,-2.568681,not reject
...,...,...,...,...,...,...,...,...
TRNFP,-1.299223,0.629528,0.0,814.0,-3.438409,-2.865097,-2.568664,not reject
UPRO,-1.543114,0.512086,5.0,811.0,-3.438439,-2.865110,-2.568671,not reject
VSMO,-2.058349,0.261589,2.0,814.0,-3.438409,-2.865097,-2.568664,not reject
VTBR,-1.106038,0.712658,15.0,803.0,-3.438520,-2.865146,-2.568690,not reject


In [54]:
results_df.loc[results_df['null hypothesis']=='reject']

,ADF Test Statistic,p-value,#Lags Used,Number of Observations Used,Critical Value (1%),Critical Value (5%),Critical Value (10%),null hypothesis
DIAS,-4.029998,0.001261,17.0,802.0,-3.438530,-2.865151,-2.568693,reject
HNFG,-3.157041,0.022610,21.0,798.0,-3.438571,-2.865169,-2.568702,reject
HYDR,-3.154677,0.022763,1.0,818.0,-3.438369,-2.865080,-2.568655,reject


### Calculate returns

In [55]:
# Calculate returns
data = df.pivot(index='TRADEDATE', columns='TICKER', values='CLOSE')
print(data.isnull().sum())
# data = data.dropna(axis=1, thresh=len(data) - 10)
data = data.fillna(method='ffill').fillna(method='bfill')
returns = np.log(data / data.shift(1)).dropna()
returns

TICKER
AFKS      1
AFLT      1
AGRO      5
AKRN      3
ALRS      1
       ... 
VKCO    250
VSMO      3
VTBR      1
WUSH    458
YNDX      4
Length: 85, dtype: int64


/tmp/ipykernel_442323/1646504256.py:5: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data = data.fillna(method='ffill').fillna(method='bfill')


TICKER,AFKS,AFLT,AGRO,AKRN,ALRS,AQUA,ASTR,BANE,BSPB,CARM,...,TRMK,TRNFP,UGLD,UPRO,VEON-RX,VKCO,VSMO,VTBR,WUSH,YNDX
TRADEDATE,,,,,,,,,,,,,,,,,,,,,
2021-02-02,0.025567,-0.002847,0.006445,0.005593,0.015191,-0.010091,0.000000,0.014258,0.011655,0.000000,...,0.001685,0.002095,0.000000,-0.005288,0.000000,0.000000,0.001835,0.000000,0.000000,0.030395
2021-02-03,0.009975,0.007669,-0.014954,0.000984,0.014674,0.010091,0.000000,0.003308,-0.001657,0.000000,...,-0.019722,0.005218,0.000000,0.006694,0.000000,0.000000,-0.001835,0.001488,0.000000,-0.016098
2021-02-04,0.009259,-0.001699,0.005159,0.003272,-0.010181,0.004008,0.000000,-0.006324,0.014631,0.000000,...,0.002743,-0.001389,0.000000,0.008043,0.000000,0.000000,0.009141,0.010218,0.000000,0.020013
2021-02-05,0.029848,0.003678,-0.001119,-0.005241,-0.002830,0.003992,0.000000,0.002414,-0.009669,0.000000,...,0.005805,0.005890,0.000000,-0.003489,0.000000,0.000000,0.002726,0.013419,0.000000,-0.008608
2021-02-08,0.016467,0.003665,0.002014,-0.000986,0.006138,0.000000,0.000000,0.015551,0.015823,0.000000,...,0.007463,0.012017,0.000000,0.011123,0.000000,0.000000,0.023319,0.010503,0.000000,0.017302
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-05-13,0.004374,0.024477,-0.020242,0.006279,0.015322,-0.002211,0.042774,0.006342,0.014742,0.007476,...,0.006943,0.004270,-0.022371,-0.007684,0.000000,0.004859,0.002534,0.000866,-0.000821,0.011594
2024-05-14,0.034926,0.039140,-0.030485,-0.008063,-0.004408,-0.002771,-0.009357,-0.001766,0.002106,0.002763,...,-0.002487,0.017198,0.007580,-0.006829,-0.010178,0.044030,-0.001519,-0.001733,0.058920,0.018614
2024-05-15,0.034437,0.046159,0.012464,-0.001668,0.005915,0.000000,-0.004985,-0.005020,0.020609,0.021416,...,-0.006423,-0.002396,0.007923,-0.016583,-0.006414,0.020984,-0.000507,0.006268,-0.025667,-0.030438


In [56]:
# List of eliminated companies 
list_returns = returns.columns
tickers = df['TICKER'].unique()

diff_list2 = [item for item in tickers if item not in list_returns]
print("List of eliminated companies:", diff_list2)

List of eliminated companies: []


### Check series stationarity

In [57]:
import pandas as pd
from statsmodels.tsa.stattools import adfuller

# Функция для выполнения теста ADF и возвращения результатов в виде Series
def adf_test(series):
    result = adfuller(series.dropna(), autolag='AIC')
    labels = ['ADF Test Statistic', 'p-value', '#Lags Used', 'Number of Observations Used']
    out = pd.Series(result[0:4], index=labels)
    for key, value in result[4].items():
        out[f'Critical Value ({key})'] = value
    return out

# Сбор всех результатов в один DataFrame
results_dict = {}

for company in returns.columns:
    results_dict[company] = adf_test(returns[company])

# Преобразование словаря результатов в DataFrame
results_df = pd.DataFrame(results_dict)

# Транспонирование DataFrame для желаемого формата (компании в столбцах)
results_df = results_df.T

# Добавление столбца с результатом гипотезы
results_df['null hypothesis'] = results_df['p-value'].apply(lambda p: 'reject' if p <= 0.05 else 'accept')

# Вывод таблицы
print(results_df)

# Вывод интерпретации результатов (опционально, если требуется дополнительное пояснение)
for company in returns.columns:
    p_value = results_df.loc[company, 'p-value']
    if p_value <= 0.05:
        print(f"{company}: Strong evidence against the null hypothesis, reject the null hypothesis. Data has no unit root and is stationary.")
    else:
        print(f"{company}: Weak evidence against the null hypothesis, time series has a unit root, indicating it is non-stationary.")



      ADF Test Statistic       p-value  #Lags Used  \
AFKS          -12.282972  8.144464e-23         3.0   
AFLT          -11.311542  1.235508e-20         8.0   
AGRO          -12.898053  4.306560e-24         5.0   
AKRN           -7.002260  7.270897e-10        18.0   
ALRS          -10.734030  2.930106e-19        10.0   
...                  ...           ...         ...   
VKCO           -5.512473  1.958091e-06        21.0   
VSMO           -9.724414  9.357018e-17         9.0   
VTBR           -6.890375  1.360505e-09        16.0   
WUSH           -6.095278  1.013582e-07        19.0   
YNDX          -12.395665  4.681538e-23         6.0   

      Number of Observations Used  Critical Value (1%)  Critical Value (5%)  \
AFKS                        815.0            -3.438399            -2.865093   
AFLT                        810.0            -3.438449            -2.865115   
AGRO                        813.0            -3.438419            -2.865102   
AKRN                        800.0  

In [58]:
results_df

,ADF Test Statistic,p-value,#Lags Used,Number of Observations Used,Critical Value (1%),Critical Value (5%),Critical Value (10%),null hypothesis
AFKS,-12.282972,8.144464e-23,3.0,815.0,-3.438399,-2.865093,-2.568662,reject
AFLT,-11.311542,1.235508e-20,8.0,810.0,-3.438449,-2.865115,-2.568674,reject
AGRO,-12.898053,4.306560e-24,5.0,813.0,-3.438419,-2.865102,-2.568667,reject
AKRN,-7.002260,7.270897e-10,18.0,800.0,-3.438551,-2.865160,-2.568697,reject
ALRS,-10.734030,2.930106e-19,10.0,808.0,-3.438469,-2.865124,-2.568678,reject
...,...,...,...,...,...,...,...,...
VKCO,-5.512473,1.958091e-06,21.0,797.0,-3.438581,-2.865173,-2.568705,reject
VSMO,-9.724414,9.357018e-17,9.0,809.0,-3.438459,-2.865119,-2.568676,reject
VTBR,-6.890375,1.360505e-09,16.0,802.0,-3.438530,-2.865151,-2.568693,reject
WUSH,-6.095278,1.013582e-07,19.0,799.0,-3.438561,-2.865164,-2.568700,reject


In [28]:
# ADF viz
# for company in returns.columns:
#     plot_acf(returns[company].dropna(), title=f'Autocorrelation for {company} Returns', lags=20)
#     plt.show()

In [59]:
returns = returns.reset_index()
returns

TICKER,TRADEDATE,AFKS,AFLT,AGRO,AKRN,ALRS,AQUA,ASTR,BANE,BSPB,...,TRMK,TRNFP,UGLD,UPRO,VEON-RX,VKCO,VSMO,VTBR,WUSH,YNDX
0,2021-02-02,0.025567,-0.002847,0.006445,0.005593,0.015191,-0.010091,0.000000,0.014258,0.011655,...,0.001685,0.002095,0.000000,-0.005288,0.000000,0.000000,0.001835,0.000000,0.000000,0.030395
1,2021-02-03,0.009975,0.007669,-0.014954,0.000984,0.014674,0.010091,0.000000,0.003308,-0.001657,...,-0.019722,0.005218,0.000000,0.006694,0.000000,0.000000,-0.001835,0.001488,0.000000,-0.016098
2,2021-02-04,0.009259,-0.001699,0.005159,0.003272,-0.010181,0.004008,0.000000,-0.006324,0.014631,...,0.002743,-0.001389,0.000000,0.008043,0.000000,0.000000,0.009141,0.010218,0.000000,0.020013
3,2021-02-05,0.029848,0.003678,-0.001119,-0.005241,-0.002830,0.003992,0.000000,0.002414,-0.009669,...,0.005805,0.005890,0.000000,-0.003489,0.000000,0.000000,0.002726,0.013419,0.000000,-0.008608
4,2021-02-08,0.016467,0.003665,0.002014,-0.000986,0.006138,0.000000,0.000000,0.015551,0.015823,...,0.007463,0.012017,0.000000,0.011123,0.000000,0.000000,0.023319,0.010503,0.000000,0.017302
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
814,2024-05-13,0.004374,0.024477,-0.020242,0.006279,0.015322,-0.002211,0.042774,0.006342,0.014742,...,0.006943,0.004270,-0.022371,-0.007684,0.000000,0.004859,0.002534,0.000866,-0.000821,0.011594
815,2024-05-14,0.034926,0.039140,-0.030485,-0.008063,-0.004408,-0.002771,-0.009357,-0.001766,0.002106,...,-0.002487,0.017198,0.007580,-0.006829,-0.010178,0.044030,-0.001519,-0.001733,0.058920,0.018614
816,2024-05-15,0.034437,0.046159,0.012464,-0.001668,0.005915,0.000000,-0.004985,-0.005020,0.020609,...,-0.006423,-0.002396,0.007923,-0.016583,-0.006414,0.020984,-0.000507,0.006268,-0.025667,-0.030438
817,2024-05-16,0.012278,-0.014734,-0.010331,-0.006700,0.013955,0.001109,-0.001485,-0.005194,0.019039,...,-0.013879,0.004786,-0.004907,0.007865,-0.002577,-0.005531,-0.001522,-0.000647,-0.004907,0.001377


In [79]:
df_feb23 = returns.loc[returns['TRADEDATE'] < '2022-02-09']
# Convert datetime column to numerical value
df_feb23['TRADEDATE'] = pd.to_datetime(df_feb23['TRADEDATE'])
df_feb23

/tmp/ipykernel_442323/242157630.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_feb23['TRADEDATE'] = pd.to_datetime(df_feb23['TRADEDATE'])


TICKER,TRADEDATE,AFKS,AFLT,AGRO,AKRN,ALRS,AQUA,ASTR,BANE,BSPB,...,TRMK,TRNFP,UGLD,UPRO,VEON-RX,VKCO,VSMO,VTBR,WUSH,YNDX
0,2021-02-02,0.025567,-0.002847,0.006445,0.005593,0.015191,-0.010091,0.0,0.014258,0.011655,...,0.001685,0.002095,0.0,-0.005288,0.000000,0.000000,0.001835,0.000000,0.0,0.030395
1,2021-02-03,0.009975,0.007669,-0.014954,0.000984,0.014674,0.010091,0.0,0.003308,-0.001657,...,-0.019722,0.005218,0.0,0.006694,0.000000,0.000000,-0.001835,0.001488,0.0,-0.016098
2,2021-02-04,0.009259,-0.001699,0.005159,0.003272,-0.010181,0.004008,0.0,-0.006324,0.014631,...,0.002743,-0.001389,0.0,0.008043,0.000000,0.000000,0.009141,0.010218,0.0,0.020013
3,2021-02-05,0.029848,0.003678,-0.001119,-0.005241,-0.002830,0.003992,0.0,0.002414,-0.009669,...,0.005805,0.005890,0.0,-0.003489,0.000000,0.000000,0.002726,0.013419,0.0,-0.008608
4,2021-02-08,0.016467,0.003665,0.002014,-0.000986,0.006138,0.000000,0.0,0.015551,0.015823,...,0.007463,0.012017,0.0,0.011123,0.000000,0.000000,0.023319,0.010503,0.0,0.017302
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
257,2022-02-02,-0.017931,0.001400,0.006419,0.006789,0.023211,-0.027815,0.0,0.010050,-0.001488,...,0.003706,-0.009779,0.0,-0.001229,-0.019480,-0.035080,-0.003364,-0.006933,0.0,-0.013245
258,2022-02-03,-0.028023,-0.026574,-0.036110,-0.059914,-0.021039,-0.043675,0.0,-0.018019,-0.020438,...,-0.009290,-0.002715,0.0,-0.011961,0.005474,-0.050213,0.003784,-0.027618,0.0,-0.043945
259,2022-02-04,0.004059,0.008581,0.001515,0.101916,0.009587,0.017700,0.0,0.001453,0.010334,...,0.025018,0.029133,0.0,0.000000,-0.002277,-0.035851,-0.002942,0.000121,0.0,-0.004160
260,2022-02-07,-0.017005,0.000000,0.014094,0.108461,-0.003099,-0.005277,0.0,0.012270,0.003005,...,0.061527,-0.009950,0.0,-0.003325,-0.029148,0.001720,-0.000421,-0.003643,0.0,-0.009870


In [81]:
df_feb23.info()

<class 'pandas.core.frame.DataFrame'>
Index: 262 entries, 0 to 261
Data columns (total 86 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   TRADEDATE  262 non-null    datetime64[ns]
 1   AFKS       262 non-null    float64       
 2   AFLT       262 non-null    float64       
 3   AGRO       262 non-null    float64       
 4   AKRN       262 non-null    float64       
 5   ALRS       262 non-null    float64       
 6   AQUA       262 non-null    float64       
 7   ASTR       262 non-null    float64       
 8   BANE       262 non-null    float64       
 9   BSPB       262 non-null    float64       
 10  CARM       262 non-null    float64       
 11  CBOM       262 non-null    float64       
 12  CHMF       262 non-null    float64       
 13  CIAN       262 non-null    float64       
 14  DIAS       262 non-null    float64       
 15  DSKY       262 non-null    float64       
 16  ELFV       262 non-null    float64       
 17  EN

In [87]:

data_slope = []
for tickers in df_feb23.columns:
    if df_feb23[tickers].dtype != 'datetime64[ns]':
        beta, alpha = np.polyfit(df_feb23['IMOEX'], df_feb23[tickers],1)
        df_slope = {'TICKER': tickers, 'beta': round(beta, 5), 'alpha': round(alpha, 5)}
        data_slope.append(df_slope)
        print('Beta for {} stock is = {} and alpha is = {}'.format('AFLT', round(beta,5), round(alpha,5)))
        
df_slope = pd.DataFrame(data_slope)
df_slope


Beta for AFLT stock is = 0.89618 and alpha is = -0.00204
Beta for AFLT stock is = 0.63574 and alpha is = -0.00093
Beta for AFLT stock is = 0.64578 and alpha is = 0.00051
Beta for AFLT stock is = 0.5345 and alpha is = 0.00332
Beta for AFLT stock is = 0.78986 and alpha is = 0.00047
Beta for AFLT stock is = 0.60792 and alpha is = 0.00306
Beta for AFLT stock is = 0.0 and alpha is = 0.0
Beta for AFLT stock is = 0.71502 and alpha is = -0.00067
Beta for AFLT stock is = 0.77763 and alpha is = 0.00143
Beta for AFLT stock is = 0.0 and alpha is = 0.0
Beta for AFLT stock is = 0.72124 and alpha is = 0.00027
Beta for AFLT stock is = 0.64587 and alpha is = 0.00078
Beta for AFLT stock is = 0.76266 and alpha is = -0.00245
Beta for AFLT stock is = 0.0 and alpha is = 0.0
Beta for AFLT stock is = 0.38485 and alpha is = -0.00131
Beta for AFLT stock is = 0.0 and alpha is = 0.0
Beta for AFLT stock is = 0.84185 and alpha is = 0.00055
Beta for AFLT stock is = 0.5728 and alpha is = -0.00171
Beta for AFLT stock 

,TICKER,beta,alpha
0,AFKS,0.89618,-0.00204
1,AFLT,0.63574,-0.00093
2,AGRO,0.64578,0.00051
3,AKRN,0.53450,0.00332
4,ALRS,0.78986,0.00047
...,...,...,...
80,VKCO,0.52388,-0.00159
81,VSMO,0.19323,0.00305
82,VTBR,0.97528,0.00030
83,WUSH,0.00000,0.00000


### Check the significants of coeff

In [95]:
# df_feb23 = df_feb23.drop(columns='TRADEDATE')
res = []
for ticker in df_feb23.columns:
    X = add_constant(df_feb23['IMOEX'])
    y = df_feb23[ticker]
    model = OLS(y, X).fit()

    params = model.params
    p_values = model.pvalues
    
    # Проверяем значимость коэффициентов
    alpha_significant = p_values['const'] < 0.05
    beta_significant = p_values['IMOEX'] < 0.05
    
    # Добавляем результаты в список
    res.append({'TICKER': ticker,
                    'alpha_significant': alpha_significant,
                    'beta_significant': beta_significant})
    print(model.summary())

# Создаем DataFrame из списка результатов
df_results = pd.DataFrame(res)

# Выводим результаты
print(df_results)

                            OLS Regression Results                            
Dep. Variable:                   AFKS   R-squared:                       0.367
Model:                            OLS   Adj. R-squared:                  0.365
Method:                 Least Squares   F-statistic:                     150.7
Date:                 Вс, 19 мая 2024   Prob (F-statistic):           1.24e-27
Time:                        22:53:49   Log-Likelihood:                 733.15
No. Observations:                 262   AIC:                            -1462.
Df Residuals:                     260   BIC:                            -1455.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0020      0.001     -2.228      0.0

/home/alexey/.local/lib/python3.10/site-packages/statsmodels/regression/linear_model.py:1782: RuntimeWarning: invalid value encountered in scalar divide
  return 1 - self.ssr/self.centered_tss
/home/alexey/.local/lib/python3.10/site-packages/statsmodels/regression/linear_model.py:1871: RuntimeWarning: invalid value encountered in scalar divide
  return self.mse_model/self.mse_resid
/home/alexey/.local/lib/python3.10/site-packages/statsmodels/regression/linear_model.py:957: RuntimeWarning: divide by zero encountered in log
  llf = -nobs2*np.log(2*np.pi) - nobs2*np.log(ssr / nobs) - nobs2
/home/alexey/.local/lib/python3.10/site-packages/statsmodels/stats/stattools.py:50: RuntimeWarning: invalid value encountered in scalar divide
  dw = np.sum(diff_resids**2, axis=axis) / np.sum(resids**2, axis=axis)
/home/alexey/.local/lib/python3.10/site-packages/statsmodels/regression/linear_model.py:1782: RuntimeWarning: invalid value encountered in scalar divide
  return 1 - self.ssr/self.centered_ts

                            OLS Regression Results                            
Dep. Variable:                   MRKU   R-squared:                       0.063
Model:                            OLS   Adj. R-squared:                  0.059
Method:                 Least Squares   F-statistic:                     17.44
Date:                 Вс, 19 мая 2024   Prob (F-statistic):           4.06e-05
Time:                        22:53:49   Log-Likelihood:                 615.95
No. Observations:                 262   AIC:                            -1228.
Df Residuals:                     260   BIC:                            -1221.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0003      0.001      0.227      0.8

/home/alexey/.local/lib/python3.10/site-packages/statsmodels/regression/linear_model.py:1782: RuntimeWarning: invalid value encountered in scalar divide
  return 1 - self.ssr/self.centered_tss
/home/alexey/.local/lib/python3.10/site-packages/statsmodels/regression/linear_model.py:1871: RuntimeWarning: invalid value encountered in scalar divide
  return self.mse_model/self.mse_resid
/home/alexey/.local/lib/python3.10/site-packages/statsmodels/regression/linear_model.py:957: RuntimeWarning: divide by zero encountered in log
  llf = -nobs2*np.log(2*np.pi) - nobs2*np.log(ssr / nobs) - nobs2
/home/alexey/.local/lib/python3.10/site-packages/statsmodels/stats/stattools.py:50: RuntimeWarning: invalid value encountered in scalar divide
  dw = np.sum(diff_resids**2, axis=axis) / np.sum(resids**2, axis=axis)
/home/alexey/.local/lib/python3.10/site-packages/statsmodels/regression/linear_model.py:1782: RuntimeWarning: invalid value encountered in scalar divide
  return 1 - self.ssr/self.centered_ts

In [99]:
# df_feb23 = df_feb23.drop(columns='TRADEDATE')
res = []
for ticker in df_feb23.columns:
    X = add_constant(df_feb23['IMOEX'])
    y = df_feb23[ticker]
    model = OLS(y, X).fit()

    params = model.params
    p_values = model.pvalues
    
    # Определяем уровень значимости для каждого коэффициента
    if p_values['const'] < 0.001:
        alpha_significance = '0.1%'
    elif p_values['const'] < 0.01:
        alpha_significance = '1%'
    elif p_values['const'] < 0.05:
        alpha_significance = '5%'
    else:
        alpha_significance = 'Not significant'
        
    if p_values['IMOEX'] < 0.001:
        beta_significance = '0.1%'
    elif p_values['IMOEX'] < 0.01:
        beta_significance = '1%'
    elif p_values['IMOEX'] < 0.05:
        beta_significance = '5%'
    else:
        beta_significance = 'Not significant'
    
    # Добавляем результаты в список
    res.append({'TICKER': ticker,
                'alpha_significance': alpha_significance,
                'beta_significance': beta_significance})
    print(model.summary())

# Создаем DataFrame из списка результатов
df_results = pd.DataFrame(res)

# Выводим результаты
print(df_results)


                            OLS Regression Results                            
Dep. Variable:                   AFKS   R-squared:                       0.367
Model:                            OLS   Adj. R-squared:                  0.365
Method:                 Least Squares   F-statistic:                     150.7
Date:                 Вс, 19 мая 2024   Prob (F-statistic):           1.24e-27
Time:                        23:15:53   Log-Likelihood:                 733.15
No. Observations:                 262   AIC:                            -1462.
Df Residuals:                     260   BIC:                            -1455.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0020      0.001     -2.228      0.0

In [100]:
df_results

,TICKER,alpha_significance,beta_significance
0,AFKS,5%,0.1%
1,AFLT,Not significant,0.1%
2,AGRO,Not significant,0.1%
3,AKRN,5%,0.1%
4,ALRS,Not significant,0.1%
...,...,...,...
80,VKCO,5%,0.1%
81,VSMO,1%,5%
82,VTBR,Not significant,0.1%
83,WUSH,Not significant,Not significant


In [11]:
df_feb23 = df_feb23[['TRADEDATE', 'AFKS', 'AFLT', 'AQUA','MOEX']]
df_feb23.to_excel('./test_camp_for_excel.xlsx')


In [17]:
df_sec = pd.read_csv('security.csv')
df_sec

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xc8 in position 176: invalid continuation byte